Na początku zamontowaliśmy dysk Google.

In [ ]:
# import google drive
from google.colab import drive

# mount google drive
drive.mount('/content/drive')


Następnie rozpakowaliśmy folder z danymi do `/content/data_dir`

In [ ]:
# unzip data.zip
!unzip "/content/drive/MyDrive/studies/ai-project-2/data.zip" -d /content/data_dir


Dalszym etapem było zaimportowanie potrzebnych bibliotek.

In [ ]:
# import
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import matplotlib.pyplot as plt
import random
import cv2
from sklearn.model_selection import train_test_split


Zapisaliśmy do zmiennej informację o ścieżce z danymi.

In [ ]:
# data_dir
data_dir = "/content/data_dir"
data_dir = pathlib.Path(data_dir)


Obliczyliśmy ilość wsyztskich zdjęć.

In [ ]:
# image_count
image_count = len(list(data_dir.glob('*/*.*')))


Ustawiliśmy `batch_size`, `img_height` i `img_width`.

In [ ]:
# configs
batch_size = 3
img_height = 128
img_width = 128


Zapisaliśmy do tablicy listę kategorii.

In [ ]:
# class names
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"]))


Zebraliśmy wszystkie ścieżki zdjęć z każdej kategorii.

In [ ]:
# imagePaths
imagePaths = []

for k, class_name in enumerate(class_names):
    for f in os.listdir(str(data_dir)+"/"+class_name):
        imagePaths.append([str(data_dir)+"/"+class_name+'/'+f, k])

random.shuffle(imagePaths)


Zmieniliśmy rozmiary zdjęć, tak żeby każde miało taki sam oraz zapisaliśmy do jednej tablicy zdjęcia, a do drugiej odpowiadające im labele.

In [ ]:
# data and labels
data = []
labels = []

for imagePath in imagePaths:
    image = cv2.imread(imagePath[0])
    image = cv2.resize(image, (img_width, img_height))
    data.append(image)
    
    label = imagePath[1]
    labels.append(label)


Jako że zdjęcia w AI/ML nie mogą być obsługiwane w skali [0, 255] na kolor, tylko w skali [0, 1], to zmieniamy je.

In [ ]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)


Wyświtalmy wykres z kilkoma przykładowymi zdjęciami.

In [ ]:
# debug
plt.subplots(3,4)
for i in range(12):
    plt.subplot(3,4, i+1)
    plt.imshow(data[i])
    plt.axis('off')
    plt.title(class_names[labels[i]])
plt.show()


Przygotowujemy dane treningowe i testowe oraz odpowiadujące im labele.

In [ ]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)


Tworzymy model `U-Net`.

In [ ]:
# create model
model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),

  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

  tf.keras.layers.Conv2D(1024, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(1024, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(512, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(512, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(256, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(256, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(128, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same'),
  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(64, (3,3), activation='elu', kernel_initializer='he_normal', padding='same'),

  tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid'),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(len(class_names))
])


Kompilujemy model przy użyciu optymalizera `adam` i metryki `accuracy`.

In [ ]:
# compile model
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


Trenujemy model na danych treningowych. Trening wykonujemy przez 3 epoki.

In [ ]:
# train model
epochs = 3
model.fit(
  trainX,
  trainY,
  epochs=epochs
)


Na danych testowych predyktujemy model, dzięki czemu otrzymujemy otrzymujemy informacje, które obrazki pasowały do jakich kategorii i z jakim powodzeniem. Następnie obliczamy ile obrazków zostało przypisanych której kategorii, z czego wyświetlamy wykres dla wizualizacji.

In [ ]:
from numpy import argmax
from sklearn.metrics import confusion_matrix, accuracy_score

pred = model.predict(testX)
predictions = argmax(pred, axis=1) # return to label

cm = confusion_matrix(testY, predictions)

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Model confusion matrix')
fig.colorbar(cax)
ax.set_xticklabels(['', *class_names])
ax.set_yticklabels(['', *class_names])

for i in range(len(class_names)):
    for j in range(len(class_names)):
        ax.text(i, j, cm[j, i], va='center', ha='center')

plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


accuracy = accuracy_score(testY, predictions)
print("Accuracy : %.2f%%" % (accuracy*100.0))